In [4]:
import os
import glob
from osgeo import gdal

In [5]:
def create_vrt_and_convert_to_cog(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    vrt_path = os.path.join(output_folder, "merged.vrt")
    output_cog_path = os.path.join(output_folder, "final_output_cog.tif")
    input_files = glob.glob(os.path.join(input_folder, "**", "*.tif"), recursive=True)

    vrt = gdal.BuildVRT(vrt_path, input_files, options=gdal.BuildVRTOptions(resampleAlg='nearest', addAlpha=False))
    vrt = None

    gdal.SetConfigOption('GDAL_CACHEMAX', '1024')
    gdal.SetConfigOption('GDAL_NUM_THREADS', 'ALL_CPUS')
    
    translate_options = gdal.TranslateOptions(
        format='COG',
        creationOptions=[
            "COMPRESS=LZW", "PREDICTOR=2", "BIGTIFF=YES", "TILED=YES",
            "BLOCKSIZE=512", "OVERVIEW_RESAMPLING=AVERAGE", "NUM_THREADS=ALL_CPUS"
        ]
    )
    
    gdal.Translate(output_cog_path, vrt_path, options=translate_options)
    os.remove(vrt_path)

In [3]:
# Example usage
input_folder = '/Users/shuyang/Data/DTM/Sudbury/Sudbury-DTM-11/DTM'
output_folder = '//Users/shuyang/Data/DTM/Sudbury/Sudbury-DTM-11'
create_vrt_and_convert_to_cog(input_folder, output_folder)

Warning 6: driver COG does not support creation option TILED


In [ ]:
def upload_to_aws(local_file, bucket, s3_folder):
    s3 = boto3.client('s3')
    
    try:
        file_name = os.path.basename(local_file)
        s3_file = os.path.join(s3_folder, file_name)
        
        print(f"Uploading {file_name} to AWS S3...")
        s3.upload_file(local_file, bucket, s3_file)
        print(f"Upload Successful. File available at: https://{bucket}.s3.amazonaws.com/{s3_file}")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False

In [ ]:
# Upload to S3
if cog_file:
    upload_to_aws(cog_file, 'ccemp-bucket', 'Public Access/COG/')
else:
    print("COG creation failed, no upload attempted.")